# 設置環境並載入套件

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute')
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute


In [2]:
import pandas as pd
import re
import unicodedata
import pickle
from collections import Counter

In [3]:
# 載入事先寫好的 functions
from compute_parameters import *

# 讀入語料

In [4]:
naer_written_path = '../CorpusforDDL/naer/naer_written'
all_files = os.listdir(naer_written_path)

In [23]:
import xml.etree.ElementTree as ET
tree = ET.parse(f'{naer_written_path}/{all_files[0]}')
root = tree.getroot()

In [24]:
naer_written_corpus = [x.text for x in root.iter('sentence')]

In [25]:
# 前處理
def preprocess_naer(string):

  clean_string = unicodedata.normalize('NFKC', string) # 全形轉半形
  clean_string = re.sub(r'\n', '', clean_string) # 移除換行符號
  clean_string = re.sub(r'\(PUNC\)', '', clean_string) # 移除標點符號標記

  clean_string = clean_string.translate(str.maketrans({',': '，', 
                                                       '!': '！', 
                                                       '?': '？',
                                                       ':': '：',
                                                       ';': '；'})) # 部分符號改回全形
  return clean_string

In [26]:
naer_written_preprocessed = []

for text in naer_written_corpus:
  preprocessed = [preprocess_naer(string) for string in text]
  joined = ''.join(preprocessed)
  split_1 = re.split(r'(?<=。」|！」|？」)', joined) # 先用 。」 ！」 ？」 分隔
  split_2 = [re.split(r'(?<=[。！？])(?!」)', x) for x in split_1] # 再用 。！？ 分隔
  split = [item for sublist in split_2 for item in sublist] # 將 list of list 攤平

  naer_written_preprocessed.append(split)

### 語料格式準備 - 1
斷詞 + pos tag

In [47]:
naer_written_ws_pos = [x[0] for x in naer_written_preprocessed]
naer_written_ws_pos[0]

'唯一(A) 切記(VE) 的(DE) 是(SHI) ，(PUNC) 千萬(D) 別(D) 用(P) 言語(Na) 或(Caa) 肢體(Na) 暴力(Na) 對(P) 他人(Nh) 發洩(VC) ，(PUNC) 因為(Cbb) 這(Nep) 將(D) 造成(VG) 情緒(Na) 的(DE) 後坐力(Na) ，(PUNC) 讓(VL) 你(Nh) 在(P) 逞(VC) 一時(Nd) 之(DE) 快(Na) 後(Ng) ，(PUNC) 湧生(Na) 更多(Neqa) 負面(Na) 情緒(Na) 。'

### 語料格式準備 - 2
斷好詞，無標點符號

In [30]:
def naer_get_sentence(sentence):

  sentence_words = re.sub(r'\([A-z0-9]+\)', '', sentence)
  sentence_words = re.sub(r'[^\w\s]', '', sentence_words).strip().split(' ')
  sentence_words = list(filter(None, sentence_words))

  return sentence_words

In [48]:
naer_written_ws = [naer_get_sentence(sentence) for sentence in naer_written_ws_pos]
naer_written_ws[0]

['唯一',
 '切記',
 '的',
 '是',
 '千萬',
 '別',
 '用',
 '言語',
 '或',
 '肢體',
 '暴力',
 '對',
 '他人',
 '發洩',
 '因為',
 '這',
 '將',
 '造成',
 '情緒',
 '的',
 '後坐力',
 '讓',
 '你',
 '在',
 '逞',
 '一時',
 '之',
 '快',
 '後',
 '湧生',
 '更多',
 '負面',
 '情緒']

### 語料格式準備 - 3
完整句子，含標點符號

In [49]:
naer_written_text = []
for sent in naer_written_ws_pos:
  res = re.sub(r'\([A-z0-9_]+\)', '', sent)
  res = res.replace(' ', '')
  naer_written_text.append(res)

naer_written_text[0]

'唯一切記的是，千萬別用言語或肢體暴力對他人發洩，因為這將造成情緒的後坐力，讓你在逞一時之快後，湧生更多負面情緒。'

# 句子長度

In [50]:
sentence_length = [len(sentence) for sentence in naer_written_ws]
sentence_length[:10]

[33, 28, 28, 20, 15, 45, 17, 24, 16, 23]

# 高頻詞與低頻詞

In [51]:
high_low_freq = [get_high_low_freq(sentence) for sentence in naer_written_ws]
high_low_freq[:10]

[{'High': 31, 'Low': 2},
 {'High': 28},
 {'High': 28},
 {'High': 20},
 {'High': 14, 'Low': 1},
 {'High': 42, 'Low': 3},
 {'High': 17},
 {'High': 24},
 {'High': 16},
 {'High': 23}]

# 詞頻

In [52]:
word_freq = [get_word_freq(sentence) for sentence in naer_written_ws]
word_freq[0]

'唯一(39149) 切記(1244) 的(16029838) 是(4642407) 千萬(13485) 別(78082) 用(377038) 言語(9150) 或(478077) 肢體(5566) 暴力(16677) 對(897964) 他人(35197) 發洩(3023) 因為(425061) 這(2114032) 將(448374) 造成(63141) 情緒(66126) 的(16029838) 後坐力(1) 讓(526314) 你(1414837) 在(3284238) 逞(943) 一時(19906) 之(875481) 快(116207) 後(422857) 湧生(44) 更多(44083) 負面(15336) 情緒(66126)'

# 詞彙等級

In [53]:
word_level = [get_word_level(sentence) for sentence in naer_written_ws]
word_level[0]

{'第5級': 9,
 '第6級': 3,
 '第1級': 8,
 '第4*級': 3,
 '第1*級': 4,
 '第3級': 1,
 'Unknown': 3,
 '第2級': 1,
 '第7級': 1}

# 詞彙長度

In [54]:
long_word_count = [get_long_word_count(sentence) for sentence in naer_written_ws]
long_word_count[:10]

[1, 3, 3, 0, 1, 4, 0, 2, 0, 1]

# 完整的句子

In [55]:
is_complete_sentence = [get_complete_sentence(sentence) for sentence in naer_written_ws_pos]
is_complete_sentence[:10]

['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']

# 完整的語境

In [56]:
is_complete_context = [get_complete_context(x) for x in naer_written_ws_pos]
is_complete_context[:10]

['Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'N', 'Y']

# Black list

In [57]:
is_blacklist = [get_blacklist(x) for x in naer_written_text]
is_blacklist[:10]

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

# Grey list

In [58]:
is_greylist = [get_greylist(x) for x in naer_written_ws_pos]
is_greylist[:10] 

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

# Make dataframe

In [60]:
naer_written_df = pd.DataFrame({'sentence': naer_written_text,
                        'sentence_preprocessed': [' '.join(sent) for sent in naer_written_ws],
                        'sentence_length': sentence_length,
                        'word_freq': word_freq,
                        'high_low_freq': high_low_freq,
                        'word_level': word_level,
                        'long_word_count': long_word_count,
                        'is_complete_sentence': is_complete_sentence,
                        'is_complete_context': is_complete_context,
                        'is_blacklist': is_blacklist,
                        'is_greylist': is_greylist})

In [61]:
naer_written_df = naer_written_df[naer_written_df['sentence_length']>0]
naer_written_df

,sentence,sentence_preprocessed,sentence_length,word_freq,high_low_freq,word_level,long_word_count,is_complete_sentence,is_complete_context,is_blacklist,is_greylist
0,唯一切記的是，千萬別用言語或肢體暴力對他人發洩，因為這將造成情緒的後坐力，讓你在逞一時之快後...,唯一 切記 的 是 千萬 別 用 言語 或 肢體 暴力 對 他人 發洩 因為 這 將 造成 ...,33,唯一(39149) 切記(1244) 的(16029838) 是(4642407) 千萬(1...,"{'High': 31, 'Low': 2}","{'第5級': 9, '第6級': 3, '第1級': 8, '第4*級': 3, '第1*...",1,Y,Y,N,N
1,情緒是影響天氣的因素之一，有意思的是，強烈的天災，如颱風、龍捲風、地震、海嘯、水災、火山爆發...,情緒 是 影響 天氣 的 因素 之 一 有意思 的 是 強烈 的 天災 如 颱風 龍捲風 地...,28,情緒(66126) 是(4642407) 影響(95840) 天氣(11782) 的(160...,{'High': 28},"{'第5級': 8, '第1級': 8, '第3級': 2, '第1*級': 1, '第4*...",3,Y,N,N,N
2,人跟大自然是一體的，隨著我們的每一次呼吸，體內的荷爾蒙與能量也不斷進入空氣中，並且影響了大自然。,人 跟 大自然 是 一體 的 隨著 我們 的 每 一 次 呼吸 體 內 的 荷爾蒙 與 能量...,28,人(1550512) 跟(260280) 大自然(10703) 是(4642407) 一體(...,{'High': 28},"{'第1級': 8, '第1*級': 3, '第4級': 6, '第6級': 2, '第5級...",3,Y,Y,N,N
3,越是焦慮、緊張、憂鬱、哀傷、恐懼的地區，就投射著越多氣候變動與天災的降臨。,越 是 焦慮 緊張 憂鬱 哀傷 恐懼 的 地區 就 投射 著 越 多 氣候 變動 與 天災 ...,20,越(69540) 是(4642407) 焦慮(16383) 緊張(27303) 憂鬱(929...,{'High': 20},"{'第3級': 2, '第1級': 5, '第5級': 5, '第2級': 1, '第6級'...",0,Y,Y,N,N
4,濃烈的情緒投射成天災，但強烈的天災又映照著人們的信念。,濃烈 的 情緒 投射成 天災 但 強烈 的 天災 又 映照 著 人們 的 信念,15,濃烈(1012) 的(16029838) 情緒(66126) 投射成(26) 天災(1141...,"{'High': 14, 'Low': 1}","{'第7級': 1, '第1級': 3, '第5級': 4, 'Unknown': 3, '...",1,Y,Y,N,N
...,...,...,...,...,...,...,...,...,...,...,...
579,也或許你將順利地升遷，但升遷並不是刻意攀緣追求的成果，而是因為你對工作有熱情，老闆才放心把更...,也 或許 你 將 順利 地 升遷 但 升遷 並 不 是 刻意 攀緣 追求 的 成果 而是 因...,34,也(1590760) 或許(60077) 你(1414837) 將(448374) 順利(1...,{'High': 34},"{'第1級': 11, '第4級': 4, '第5級': 1, 'Unknown': 5, ...",0,Y,Y,N,N
580,那也就是說，即使沒有薪水，你仍喜歡手上的這份工作，並且在工作的同時完成自己的價值。,那 也 就是說 即使 沒有 薪水 你 仍 喜歡 手 上 的 這 份 工作 並且 在 工作 的...,24,那(754138) 也(1590760) 就是說(5040) 即使(68376) 沒有(59...,{'High': 24},"{'第1級': 12, 'Unknown': 1, '第4*級': 3, '第4級': 3,...",1,Y,Y,N,N
581,那就好像一隻蜘蛛，牠認真地吐絲結網，而在編織絲網的當下，完成了蜘蛛的天賦，因此，結成一張八卦...,那 就 好 像 一 隻 蜘蛛 牠 認真 地 吐 絲 結網 而 在 編織 絲網 的 當下 完成...,81,那(754138) 就(1737544) 好(448428) 像(289494) 一(370...,"{'High': 77, 'Low': 4}","{'第1級': 15, 'Unknown': 20, '第1*級': 12, '第2級': ...",1,Y,Y,N,N
582,而當第一次有蜜蜂掉進蜘蛛網、成為蜘蛛的食物時，蜘蛛會覺得相當驚奇，原來牠所熱愛的蜘蛛網竟還有...,而 當 第一 次 有 蜜蜂 掉進 蜘蛛網 成為 蜘蛛 的 食物 時 蜘蛛 會 覺得 相當 驚...,31,而(937923) 當(294625) 第一(172544) 次(331701) 有(221...,{'High': 31},"{'第5級': 5, 'Unknown': 7, '第2級': 4, '第1級': 5, '...",2,Y,Y,N,N


In [62]:
naer_written_df.to_csv('../results/naer_written/naer_written_parameters_example.csv', index = False)

## Concordance

In [63]:
from nltk.text import Text

In [64]:
target_words = ['難得', '畢竟', '的確', '難免', '總是', '有助於']
corpus = [item for sublist in naer_written_ws for item in sublist]
text = Text(corpus)
dfs = []

for word in target_words:

  con_list = text.concordance_list(word)
  right_word = [x.right[0] for x in con_list]
  left_word = [x.left[-1] for x in con_list]
  context = [x.left + [word] + x.right for x in con_list]
  context = [' '.join(x) for x in context]

  df = pd.DataFrame({'left_word': left_word,
                     'target_word': word,
                     'right_word': right_word,
                     'context': context})
  dfs.append(df)

In [65]:
concordance_df = pd.concat(dfs)
concordance_df.to_csv('../results/naer_written/naer_written_concordance_df.csv', index = False)